last update: 2021-09-06

In [22]:
import pandas as pd
import numpy as np
import pathlib

In [2]:
src = pathlib.Path().cwd().parent.parent / 'data' / 'interim'
ref = pathlib.Path().cwd().parent.parent / 'reference'

In [3]:
df = pd.read_csv(src / 'public-data.csv')
pop = pd.read_csv(ref / 'pop_1.csv', sep=',', dtype={'id_addiv': 'str'})
addiv = pd.read_csv(ref / 'addiv.csv', sep=',', dtype={'id_addiv': 'str', 'of_addiv': 'str'})


In [4]:
df['date_report'] = pd.to_datetime(df['date_report'])

In [9]:
df.head()
# df.info()
# df.addr_prov_home.unique()

,name_full,sex,yob,addr_prov_home,addr_dist_home,addr_ward_home,addr_home,date_report,date_positive,place_recognize
0,BÙI THỊ THU,NU,1967.0,79,776,27427.0,"19D BINH DUC MEDIC , PHUONG 15",2021-07-10,2021-07-09,THIEU THONG TIN
1,BÙI VĂN TIẾN,NAM,1991.0,79,786,27643.0,2056/5A NHA BE,2021-07-10,2021-07-08,SANG LOC TAI BV
2,ĐẶNG HOÀNG PHƯỚC,NAM,1994.0,79,767,27028.0,307/43 THACH LAM,2021-07-10,2021-07-08,KHU CACH LY
3,ĐẶNG NGỌC,NU,1964.0,79,773,27292.0,88/38/7 NGUYEN KHOAI,2021-07-10,2021-07-08,KHU PHONG TOA
4,ĐẶNG THỊ CÚC,NU,1989.0,79,768,27073.0,102/35 HO BIEU CHANH,2021-07-11,2021-07-05,SANG LOC TAI BV


In [29]:
def get_no_event(data, date_col='date_report', no_col='no_test', pop=10000000, rolling=7):
    """
    Count number of (no) event by date
    
    Args:
        data: Data frame, input data frame
        date_col: String, name of date column, default 'date_report'
        no_col: String, name of event count column, default 'no_case'
        rolling: Int, rolling window, default 7
        pop: Int, population, default 10000000
    
    Return:
        A data frame
    """
    # number of events by date, reindex to fill missing date entry with 0
    df = (
        data[[date_col]]
        .groupby(date_col)
        .apply(lambda x: len(x))
        .to_frame(name=no_col)
        .reindex(pd.date_range(start=data[date_col].min(), end=data[date_col].max(), freq='D'))
        .fillna(0)
    )
    # population column
    df['pop'] = pop
    # name of columns to be created
    col_per_pop = no_col + '_ppop'
    col_rollmean = no_col + '_rollmean' + str(rolling) + 'd'
    col_per_pop_rollmean = no_col + '_ppop_rollmean' + str(rolling) + 'd'
    col_cumsum = no_col + '_cumsum'
    col_per_pop_cumsum = no_col + '_ppop_cumsum'
    # number of events per population by date
    df[col_per_pop] = round(df[no_col] /df['pop'] * 100000, 3)
    # rolling mean number of events by date
    df[col_rollmean] = df[no_col].rolling(rolling).mean()
    # rolling mean number of events per population by date
    df[col_per_pop_rollmean] = df[col_per_pop].rolling(7).mean()
    # cumulative sum number of events by date
    df[col_cumsum] = df[no_col].cumsum()
    # cumulative sum number of events per population by date
    df[col_per_pop_cumsum] = df[col_per_pop].cumsum()
    df = df.drop(columns='pop')
    return df

In [32]:
pop_total = int(pop[pop.id_addiv == '79']['pop'][0])
no_case = get_no_case(data=df, pop=pop_total,
                      date_col='date_report', no_col='no_case', rolling=7)

In [34]:
# no_case

In [44]:
def get_no_case_by_group(data, group_col, date_col='date_report', no_col='no_case',
                         available_pop=False, getname=False, rolling=7):
    """
    Count number of (no) event by group and date
    
    Args:
        data: Data frame, input data frame
        date_col: String, name of date column, default date_col
        no_col: String, name of event count column, default no_col
        rolling: Int, rolling window, default 7
        available_pop: Boolean, if population data frame by group is available, default False
        getname: Boolean, to get group label if it exists, default False
    
    Return:
        A data frame
    """
    unique = data[group_col].unique()
    # number of events by group and date
    df_1 = (
        data[[date_col, group_col]]
        .groupby([date_col, group_col])
        .apply(lambda x: len(x))
        .to_frame(name=no_col)
        .unstack(fill_value=0)
        .asfreq('D', fill_value=0)
        .stack()
        .sort_index(level=0)
        .reset_index()
    )
    # pivot table to fill missing group with 0
    df_pv = (
        df_1[[date_col, group_col, no_col]]
        .pivot(index=date_col, columns=group_col, values=no_col)
    ).fillna(0)
    # fill missing date with 0
    df_2 = (
        df_pv.reindex(pd.date_range(
            start=df_pv.index.min(),
            end=df_pv.index.max(),
            freq='D'))
        .fillna(0)
        .stack(list(range(0, 1)))
        .reset_index()
        .rename(columns={
            'level_0': date_col,
            0: no_col
        })
#         .set_index([date_col] + group)
    )
    
    # name of columns to be created
    col_per_pop = no_col + '_ppop'
    col_rollmean = no_col + '_rollmean' + str(rolling) + 'd'
    col_per_pop_rollmean = no_col + '_ppop_rollmean' + str(rolling) + 'd'
    col_cumsum = no_col + '_cumsum'
    col_per_pop_cumsum = no_col + '_ppop_cumsum'
    
    # get no_case_rollmean
    df_3 = df_2
    df_3[col_rollmean] = (
        df_3.groupby(group_col)[no_col]
        .transform(lambda x: x.rolling(rolling).mean())
    )
    # get no_case cumsum
    df_3[col_cumsum] = (
        df_3[[date_col, group_col, no_col]]
        .groupby([date_col, group_col]).sum()
        .groupby(level=1).cumsum().reset_index()
        [no_col]
    )
    
    # Get number of events per population
    if available_pop:
        df_pop = pop[['id_addiv', 'pop']]
        # merge with population
        df_3 = (
            df_3.merge(
                df_pop,
                how= 'left',
                left_on=group_col,
                right_on= 'id_addiv')
            .drop(columns=['id_addiv'])
        )
        # number of events per population by group
        df_3[col_per_pop] = round(df_3[no_col] / df_3['pop'] * 100000, 3)
        df_3 = df_3.drop(columns='pop')
        # rolling mean number of events per population by group
        df_3[col_per_pop_rollmean] = (
            df_3.groupby(group_col)[col_per_pop]
            .transform(lambda x: x.rolling(rolling).mean())
        )
        # cumulative sum number of events per population by group
        df_3[col_per_pop_cumsum] = (
            df_3[[date_col, group_col, col_per_pop]]
            .groupby([date_col, group_col]).sum()
            .groupby(level=1).cumsum().reset_index()
            [col_per_pop]
        )

    # Get district name (optional)
    if getname:
        df_3 = (
            df_3.merge(
                addiv[['id_addiv', 'name_addiv_2']],
                how = 'left',
                left_on = group_col,
                right_on = 'id_addiv'
            )
            .drop(columns=[group_col, 'id_addiv'])
            .rename(columns={'name_addiv_2': group_col})
        )
    return df_3

In [51]:
# v1
def get_no_event_by_group(data,
                         group_col,
                         date_col='date_report',
                         no_col='no_test',
                         available_pop=False,
                         getname=False, rolling=7):
    """
    Count number of (no) event by group and date
    
    Args:
        data: Data frame, input data frame
        date_col: String, name of date column, default date_col
        no_col: String, name of event count column, default no_col
        rolling: Int, rolling window, default 7
        available_pop: Boolean, if population data frame by group is available, default False
        getname: Boolean, to get group label if it exists, default False
    
    Return:
        A data frame
    """
    unique = data[group_col].unique()
    # number of events by group and date
    df_1 = (
        data[[date_col, group_col]]
        .groupby([date_col, group_col])
        .apply(lambda x: len(x))
        .to_frame(name=no_col)
        .unstack(fill_value=0)
        .asfreq('D', fill_value=0)
        .stack()
        .sort_index(level=0)
        .reset_index()
    )
    # pivot table to fill missing group with 0
    df_pv = (
        df_1[[date_col, group_col, no_col]]
        .pivot(index=date_col, columns=group_col, values=no_col)
    ).fillna(0)
    # fill missing date with 0
    df_2 = (
        df_pv.reindex(pd.date_range(
            start=df_pv.index.min(),
            end=df_pv.index.max(),
            freq='D'))
        .fillna(0)
        .stack(list(range(0, 1)))
        .reset_index()
        .rename(columns={
            'level_0': date_col,
            0: no_col
        })
#         .set_index([date_col] + group)
    )
    
    # name of columns to be created
    col_per_pop = no_col + '_ppop'
    col_rollmean = no_col + '_rollmean' + str(rolling) + 'd'
    col_per_pop_rollmean = no_col + '_ppop_rollmean' + str(rolling) + 'd'
    col_cumsum = no_col + '_cumsum'
    col_per_pop_cumsum = no_col + '_ppop_cumsum'
    
    # get no_case_rollmean
    df_3 = df_2
    df_3[col_rollmean] = (
        df_3.groupby(group_col)[no_col]
        .transform(lambda x: x.rolling(rolling).mean())
    )
    # get no_case cumsum
    df_3[col_cumsum] = (
        df_3[[date_col, group_col, no_col]]
        .groupby([date_col, group_col]).sum()
        .groupby(level=1).cumsum().reset_index()
        [no_col]
    )
    
    # Get number of events per population
    if available_pop:
        df_pop = pop[['id_addiv', 'pop']]
        # merge with population
        df_3 = (
            df_3.merge(
                df_pop,
                how= 'left',
                left_on=group_col,
                right_on= 'id_addiv')
            .drop(columns=['id_addiv'])
        )
        # number of events per population by group
        df_3[col_per_pop] = round(df_3[no_col] / df_3['pop'] * 100000, 3)
        df_3 = df_3.drop(columns='pop')
        # rolling mean number of events per population by group
        df_3[col_per_pop_rollmean] = (
            df_3.groupby(group_col)[col_per_pop]
            .transform(lambda x: x.rolling(rolling).mean())
        )
        # cumulative sum number of events per population by group
        df_3[col_per_pop_cumsum] = (
            df_3[[date_col, group_col, col_per_pop]]
            .groupby([date_col, group_col]).sum()
            .groupby(level=1).cumsum().reset_index()
            [col_per_pop]
        )

    # Get district name (optional)
    if getname:
        df_3 = (
            df_3.merge(
                addiv[['id_addiv', 'name_addiv_2']],
                how = 'left',
                left_on = group_col,
                right_on = 'id_addiv'
            )
            .drop(columns=[group_col, 'id_addiv'])
            .rename(columns={'name_addiv_2': group_col})
        )
    return df_3


In [97]:
# v2
def get_no_event_by_group(data,
                          group_col = [],
                          date_col='date_report',
                          no_col='no_event'):
    """
    Count number of (no) event by group and date
    
    Args:
        data: Data frame, input data frame
        group_col: list of group columns
        date_col: String, name of date column, default date_col
        no_col: String, name of event count column, default no_col
    
    Return:
        A data frame
    """
    g = data.groupby(group_col + [date_col])
    df = (
        data.groupby(group_col + [date_col])
        .apply(lambda x: len(x)).to_frame(name=no_col)
        .reset_index()
        .set_index(date_col)
        .groupby(group_col)
        [no_col]
        .resample('D').sum()
        .reset_index()
        .pivot(index=date_col, columns=group_col, values=no_col)
        .fillna(0)
        .stack(list(range(len(group_col))))
        .reset_index()
        .rename(columns={0: no_col})
    )
    return df

In [98]:
foo = get_no_event_by_group(data=df, date_col='date_report', no_col='no_case',
                            group_col = ['addr_dist_home', 'sex'])
# foo.set_index('date_report', inplace=True)
foo

,date_report,addr_dist_home,sex,no_case
0,2021-05-27,760,NAM,0.0
1,2021-05-27,760,NAN,0.0
2,2021-05-27,760,NU,0.0
3,2021-05-27,761,NAM,3.0
4,2021-05-27,761,NAN,0.0
...,...,...,...,...
7663,2021-09-11,786,NU,153.0
7664,2021-09-11,787,NAM,14.0
7665,2021-09-11,787,NU,10.0
7666,2021-09-11,UNKN,NAM,17.0


In [82]:
g = foo.groupby(['addr_dist_home', 'sex', 'addr_ward_home'])
baz1 = (
    g['no_case']
    .resample('D').sum()
    .reset_index()
    .pivot(index='date_report', columns=['addr_dist_home', 'sex', 'addr_ward_home'], values='no_case')
    .fillna(0)
    .stack([0, 1, 2])
    .reset_index()
    .rename(columns={0: 'no_case'})
)

In [92]:
list(range(len(group_col) + 1))

[0, 1]